In [1]:
import pandas as pd
from snorkel.labeling import labeling_function

In [10]:
responses = pd.read_csv('../data/moralmachine/SharedResponses.csv', skiprows=0, nrows=2000)
responses.describe()

,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,DefaultChoiceIsOmission,NumberOfCharacters,DiffNumberOFCharacters,Saved,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
count,1.999000e+03,2000.000000,2000.0,2000.000000,2000.00000,2000.000000,1794.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,3.089403e+15,6.778000,0.0,0.425000,0.22150,0.703000,0.496656,3.112500,0.548500,0.472500,...,0.158500,0.057500,0.10450,0.123000,0.196500,0.198000,0.102500,0.088000,0.166500,0.162500
std,3.297890e+15,3.797094,0.0,0.494467,0.41536,0.861488,0.500128,1.457019,1.126624,0.499368,...,0.435284,0.259282,0.34444,0.374087,0.510899,0.530034,0.343588,0.339579,0.554014,0.569435
min,7.302590e+06,1.000000,0.0,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.867327e+09,3.000000,0.0,0.000000,0.00000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.921975e+15,7.000000,0.0,0.000000,0.00000,0.000000,0.000000,3.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.973633e+15,10.000000,0.0,1.000000,0.00000,2.000000,1.000000,5.000000,0.000000,1.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.992829e+15,13.000000,0.0,1.000000,1.00000,2.000000,1.000000,5.000000,4.000000,1.000000,...,3.000000,2.000000,3.00000,3.000000,4.000000,5.000000,2.000000,3.000000,5.000000,5.000000


In [37]:
# For viewing the possible values for each feature
for feature in responses.columns:
    print(feature)
    print(responses[feature].unique())

ResponseID
['2222bRQqBTZ6dLnPH' '2222sJk4DcoqXXi98' '2223CNmvTr2Coj4wp' ...
 '22BSSc45dijDan7fD' '22BSb5vzyoeu2csFZ' '22BSp55p9c9LGeibw']
ExtendedSessionID
['32757157_6999801415950060.0' '1043988516_3525281295.0'
 '-1613944085_422160228641876.0' ... '1703715954_8686429570804792.0'
 '-2057390477_171517308543305.0' '-610773260_4701943483744748.0']
UserID
[6.99980142e+15 3.52528130e+09 4.22160229e+14 ... 8.68642957e+15
 1.71517309e+14 4.70194348e+15]
ScenarioOrder
[ 7  2 10 11  8  6 12  3  4  1  5 13  9]
Intervention
[0]
PedPed
[0 1]
Barrier
[0 1]
CrossingSignal
[1 0 2]
AttributeLevel
['Fit' 'Rand' 'Female' 'Old' 'More' 'Low' 'Hoomans' 'Less' 'Male' 'Pets'
 'Fat' 'Young' 'High']
ScenarioTypeStrict
['Fitness' 'Random' 'Gender' 'Age' 'Utilitarian' 'Social Status' 'Species']
ScenarioType
['Fitness' 'Random' 'Gender' 'Age' 'Utilitarian' 'Social Status' 'Species']
DefaultChoice
['Fit' nan 'Male' 'Young' 'More' 'High' 'Hoomans']
NonDefaultChoice
['Fat' nan 'Female' 'Old' 'Less' 'Low' 'Pets']
De

In [30]:
# What does a single user's session look like?
responses['UserID'].sample().values

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(responses[responses['ExtendedSessionID'] == responses['ExtendedSessionID'].sample().values[0]])

            ResponseID    ExtendedSessionID       UserID  ScenarioOrder  \
167  222jhd8yEKXJc3EsZ  713958586_944956750  944956750.0              6   

     Intervention  PedPed  Barrier  CrossingSignal AttributeLevel  \
167             0       0        0               2          Young   

    ScenarioTypeStrict ScenarioType DefaultChoice NonDefaultChoice  \
167                Age          Age         Young              Old   

     DefaultChoiceIsOmission  NumberOfCharacters  DiffNumberOFCharacters  \
167                      1.0                   1                       0   

     Saved Template  DescriptionShown  LeftHand UserCountry3  Man  Woman  \
167      1  Desktop               0.0       1.0          IDN    0      1   

     Pregnant  Stroller  OldMan  OldWoman  Boy  Girl  Homeless  LargeWoman  \
167         0         0       0         0    0     0         0           0   

     LargeMan  Criminal  MaleExecutive  FemaleExecutive  FemaleAthlete  \
167         0         0         

In [27]:
# survey = pd.read_csv('../data/moralmachine/SharedResponsesSurvey.csv', skiprows=0, nrows=500)
# survey

In [5]:
# survey_first = pd.read_csv('../data/moralmachine/SharedResponsesFullFirstSessions.csv', skiprows=0, nrows=500)
# survey_first